## Introduction

The purpose of this notebook is running Bio-inspired clustering to see if the algorithm can sucessfully cluster the news in to 'Real' & 'Fake' using just the words in the articles

## Imports

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats

import matplotlib.pyplot as plt #  plotting and data visualization
import seaborn as sns # improve visuals
sns.set() # Set as default style

import string # python library
import re # regex library

from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation, strip_multiple_whitespaces, strip_numeric, remove_stopwords, strip_short # Preprocesssing
from gensim.models import Word2Vec # Word2vec

from sklearn import cluster # Kmeans clustering
from sklearn import metrics # Metrics for evaluation
from sklearn.decomposition import PCA #PCA
from sklearn.manifold import TSNE #TSNE
import random

## Data Analysis & Cleanup

In [ ]:
fake = pd.read_csv('../input/fake-and-real-news-dataset/Fake.csv')
true = pd.read_csv('../input/fake-and-real-news-dataset/True.csv')

In [ ]:
fake.head(10)

In [ ]:
true.head(10)

The first issue as seen above is that the True data contains:

1. A reuters disclaimer that the article is a tweet
> "The following statements were posted to the verified Twitter accounts of U.S. President Donald Trump, @realDonaldTrump and @POTUS.  The opinions expressed are his own. Reuters has not edited the statements or confirmed their accuracy.  @realDonaldTrump"


2. City Name and then publisher at the start
> WASHINGTON (Reuters)

so in the next block of code I remove this from the data

In [ ]:
# The following is a crude way to remove the @realDonaldTrump tweet disclaimer and State/Publisher at start of text

cleansed_data = []
for data in true.text:
    if "@realDonaldTrump : - " in data:
        cleansed_data.append(data.split("@realDonaldTrump : - ")[1])
    elif "(Reuters) -" in data:
        cleansed_data.append(data.split("(Reuters) - ")[1])
    else:
        cleansed_data.append(data)

true["text"] = cleansed_data
true.head(10)

In [ ]:
true.text[7]

Some of the text still contains various characters/words such as:

1. Links
2. Timestamps
3. Brackets
4. Numbers

So we will be removing all such characters from the real and fake data using genlib preprocessing and a custom regex for the links in preperation for the Word2Vec

Before that however, the title and text will be merged in to one so that it can all be preprocessed together. I will also add a label for real and fake which will be used later to evaluate our clustering

In [ ]:
# Merging title and text
fake['Sentences'] = fake['title'] + ' ' + fake['text']
true['Sentences'] = true['title'] + ' ' + true['text']

# Adding fake and true label
fake['Label'] = 0
true['Label'] = 1

# We can merge both together since we now have labels
final_data = pd.concat([fake, true])

# Randomize the rows so its all mixed up
final_data = final_data.sample(frac=1).reset_index(drop=True)

# Drop columns not needed
final_data = final_data.drop(['title', 'text', 'subject', 'date'], axis = 1)

final_data.head(10)

In [ ]:
# Here we preprocess the sentences
def remove_URL(s):
    regex = re.compile(r'https?://\S+|www\.\S+|bit\.ly\S+')
    return regex.sub(r'',s)

# Preprocessing functions to remove lowercase, links, whitespace, tags, numbers, punctuation, strip words
CUSTOM_FILTERS = [lambda x: x.lower(), strip_tags, remove_URL, strip_punctuation, strip_multiple_whitespaces, strip_numeric, remove_stopwords, strip_short]

# Here we store the processed sentences and their label
processed_data = []
processed_labels = []

for index, row in final_data.iterrows():
    words_broken_up = preprocess_string(row['Sentences'], CUSTOM_FILTERS)
    # This eliminates any fields that may be blank after preprocessing
    if len(words_broken_up) > 0:
        processed_data.append(words_broken_up)
        processed_labels.append(row['Label'])

## Word2Vec

In [ ]:
# Word2Vec model trained on processed data
model = Word2Vec(processed_data, min_count=1)

In [ ]:
model.wv.most_similar("country")

## Sentence Vectors

In [ ]:
# Getting the vector of a sentence based on average of all the word vectors in the sentence
# We get the average as this accounts for different sentence lengths

def ReturnVector(x):
    try:
        return model[x]
    except:
        return np.zeros(100)
    
def Sentence_Vector(sentence):
    word_vectors = list(map(lambda x: ReturnVector(x), sentence))
    return np.average(word_vectors, axis=0).tolist()

X = []
for data_x in processed_data:
    X.append(Sentence_Vector(data_x))

In [ ]:
X_np = np.array(X)
X_np.shape

## Clustering

In [ ]:
eps0=2e-2    # learning rate
hid=2   # number of hidden units that are displayed in Ky by Kx array
mu=0.0
sigma=1.0
Nep=20      # number of epochs
Num=10      # size of the minibatch
prec=1e-30
delta=0   # Strength of the anti-hebbian learning
p=2.0        # Lebesgue norm of the weights
k=2          # ranking parameter, must be integer that is bigger or equal than 2
a = 0.4      # activity, between 0 and 1

In [ ]:
#hid is number of clusters
def train(X,hid = hid,Num= Num,Nep=Nep,delta = delta,k=k, mu=mu,sigma=sigma,eps0=eps0,prec = prec):
  M = X.copy()  
  mu = np.mean(M,axis=0)
  M = M-mu
  N = M.shape[1]
  Ns = M.shape[0]
  synapses = M[random.sample(range(Ns), hid),:]
  synapses = synapses/np.reshape(np.linalg.norm(synapses,axis=1),(synapses.shape[0],1))
  for nep in range(Nep):
      eps=eps0*(1-nep/Nep)
      M=M[np.random.permutation(Ns),:]
      for i in range(Ns//Num):
          inputs=np.transpose(M[i*Num:(i+1)*Num,:])
          sig=np.sign(synapses)
          tot_input=np.dot(sig*np.absolute(synapses)**(p-1),inputs)
          
          y=np.argsort(tot_input,axis=0)
          yl=np.zeros((hid,Num))
          yl[y[hid-1,:],np.arange(Num)]=1.0
          yl[y[hid-k],np.arange(Num)]=-delta
          
          xx=np.sum(np.multiply(yl,tot_input),1)
          ds=np.dot(yl,np.transpose(inputs)) - np.multiply(np.tile(xx.reshape(xx.shape[0],1),(1,N)),synapses)
          
          nc=np.amax(np.absolute(ds))
          if nc<prec:
              nc=prec
          synapses += eps*np.true_divide(ds,nc)
          synapses = synapses/np.reshape(np.linalg.norm(synapses,axis=1),(synapses.shape[0],1))
  return synapses,mu        

In [ ]:
n_clus = 2
synapses,x_mu = train(X_np,hid = n_clus)
clustered = np.argmax(np.dot(X_np-x_mu,synapses.T),axis = 1)

In [ ]:
def predict(X_np,synapses,x_mu,mapping):
    map_func = lambda x:mapping[x]
    clustered = np.argmax(np.dot(X_np-x_mu,synapses.T),axis = 1)
    return list(map(map_func,clustered))

## Labeling the cluster
Now that we have created 2 clusters of news, we need to properly label them into fake and real

In [ ]:
X_np.shape

In [ ]:
alpha = 0.1
n_top = int(alpha*X_np.shape[0])
true_top_lab = np.where(np.array(processed_labels[:n_top])==1)
fake_top_lab = np.where(np.array(processed_labels[:n_top])==0)

In [ ]:
true_clust_label =stats.mode(np.argmax(np.dot(X_np[true_top_lab]-x_mu,synapses.T),axis = 1))
fake_clust_label =stats.mode(np.argmax(np.dot(X_np[fake_top_lab]-x_mu,synapses.T),axis = 1))

In [ ]:
mapping = {true_clust_label[0][0]:1,fake_clust_label[0][0]:0}
map_func = lambda x:mapping[x]

In [ ]:
print(mapping)

In [ ]:
mapped_clustered = list(map(map_func,clustered))

In [ ]:
testing_df = {'Sentence': processed_data, 'Labels': processed_labels, 'Prediction': mapped_clustered}
testing_df = pd.DataFrame(data=testing_df)
testing_df.head(10)

In [ ]:
correct = 0
incorrect = 0
for index, row in testing_df.iterrows():
    if row['Labels'] == row['Prediction']:
        correct += 1
    else:
        incorrect += 1
        
print("Correctly clustered news using mean shifted Bio clustering: " + str((correct*100)/(correct+incorrect)) + "%")

The results above show that its correctly clustered them in some cases where 0 is fake news and 1 is real news

## Visualization

In [ ]:
# PCA of sentence vectors
pca = PCA(n_components=2)
pca_result = pca.fit_transform(X_np)

PCA_df = pd.DataFrame(pca_result)
PCA_df['cluster'] = clustered
PCA_df.columns = ['x1','x2','cluster']

In [ ]:
# Plots
plt.figure(2,figsize=(10,10));
sns.scatterplot(data=PCA_df,x='x1',y='x2',hue='cluster',legend="full",alpha=0.5);
plt.title('Visualized on PCA');

## Custom News Tests

In [ ]:
# Testing with fake news generated from https://www.thefakenewsgenerator.com/
onion_data = "Flint Residents Learn To Harness Superpowers, But Trump Gets Away Again They developed superpowers after years of drinking from a lead-poisoned water supply. But just having incredible abilities doesn't make them superheroes. Not yet. Donald Trump faced off against the superpowered civilians but he got away before they could catch him"

# Preprocess article
onion_data = preprocess_string(onion_data, CUSTOM_FILTERS)

# Get sentence vector
onion_data = Sentence_Vector(onion_data)

# Get prediction
predict(np.array([onion_data]),synapses,x_mu,mapping)

In [ ]:
# News from BBC

bbc_data = "Nasa Mars 2020 Mission's MiMi Aung on women in space Next year, Nasa will send a mission to Mars. The woman in charge of making the helicopter that will be sent there – which is set to become the first aircraft to fly on another planet – is MiMi Aung. At 16, MiMi travelled alone from Myanmar to the US for access to education. She is now one of the lead engineers at Nasa. We find out what it's like being a woman in space exploration, and why her mum is her biggest inspiration."

# Preprocess article
bbc_data = preprocess_string(bbc_data, CUSTOM_FILTERS)

# Get sentence vector
bbc_data = Sentence_Vector(bbc_data)

# Get prediction
predict(np.array([bbc_data]),synapses,x_mu,mapping)